# Imports

In [1]:
import numpy as np
import re
import nltk
import math
import json
import statistics
import os
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [5]:
current_directory=os.getcwd()
corpus_path = os.path.join(current_directory, 'files\CACM\cacm.txt')
stoplist_path = os.path.join(current_directory, 'files\CACM\common_words.txt')
query_path = os.path.join(current_directory, 'files\CACM\query.txt')
qrels_path = os.path.join(current_directory, 'files\CACM\qrels.txt')

# Data preparation

In [6]:
def apply_regex(regex , string):
  #Apply a regex to the input string
  matches = list(re.finditer(regex, string))
  if (len(matches) != 0):
    for match in matches:
      result = match.groups()[0]
  else:
    result = ''
  return result


def make_corpus(pathfile):
  #Construct a list in which each element contains the title, author, and abstract of the corpus documents
  f = open(pathfile , 'r')
  text = f.read()
  tab = re.split(pattern="\.I [0-9]+\n", string=text)
  tab.remove('')
  regex_title =  r"\.T\n((.|\n)*?)\.[WBAKCNX]\n"
  regex_author = r"\.A\n((.|\n)*?)\.[WBTKCNX]\n"
  regex_abstract = r"\.W\n((.|\n)*?)\.[ABTKCNX]\n"
  result = []
  for string in tab:
    title = apply_regex(regex_title , string)
    author = apply_regex(regex_author , string)
    abstract = apply_regex(regex_abstract , string)
    concat = title + ' ' + author + ' ' + abstract
    result.append(concat)
  return result


def vec2dict(tab):
  #Transform the list of documents into a dictionary
  keys = ['doc_'+str(i) for i in range(1, len(tab)+1)]
  dico = {k : v for k,v in zip(keys , tab)}
  return dico


def preprocessing(string, path_stoplist):
  #Romove commun words from sentences and transform the words into their stem form
  f = open(path_stoplist , 'r')
  local_stoplist = f.read().splitlines()
  string = re.sub("[\W]", ' ', string)
  string = string.lower()
  string = string.split()
  ps = PorterStemmer()
  string = [ps.stem(word) for word in string if (not word in set(stopwords.words('english'))) & (not word in set(local_stoplist))]
  string = ' '.join(string)
  return string


def count_freq(string):
  # Return the words contained in a sentence with their frequence in a dictinary format
  string = string.split()
  freqs = dict(FreqDist(string))
  return freqs


def treat_dataset(path_corpus, path_stoplist):
  #Transform the input corpus into a dictionary of documents IDs and content, in which each elemnt consists of the words contained in the document and their frequency of aparition
  result = make_corpus(path_corpus)
  dico = vec2dict(result)
  for key in dico:
    val = dico.get(key)
    val = preprocessing(val , path_stoplist)
    val = count_freq(val)
    dico[key] = val
  return dico

In [9]:
Data = treat_dataset(corpus_path , stoplist_path)

#Save data in a json file
json.dump( Data, open( "files\Data.json", 'w' ) )

#Data = json.load( open( os.path.join(current_directory, "files\Data.json") ) )

print('Number of documents: ',len(Data))
print(Data['doc_39'])

Number of documents:  3204
{'secant': 2, 'method': 2, 'simultan': 2, 'nonlinear': 1, 'equat': 2, 'wolf': 1, 'procedur': 1, 'solut': 1, 'system': 1, 'necessarili': 1, 'linear': 1, 'gener': 1, 'singl': 1, 'function': 1, 'variabl': 1}


# Boolean Model

In [10]:
def inverse_doc_freq(dictio):
  dictio_inverse = {}
  for doc in dictio.keys():
    info = dictio.get(doc)
    for word in info.keys():
      freq = info.get(word)
      dictio_inverse[(word , doc)] = freq
  return dictio_inverse


def inverted_index(dictio_inverse):
  access = {}
  for item in dictio_inverse:
    word = item[0]
    doc = item[1]
    freq = dictio_inverse.get(item)
    if not word in access:
      access[word] = {doc : freq}
    else:
      dictio_tmp = access.get(word)
      dictio_tmp[doc] = freq
      access[word] = dictio_tmp
  return access

In [12]:
def space(query,ch1,ch2):
  #Add space in the input query
  result=""
  lenght=len(query)
  t=0
  for ch in query:
    if t==lenght-1: result = result + ch; break
    else:
      if ch==ch1:
        result = result + ch + ' '
      else:
        if query[t+1]==ch2:
          result = result + ch + ' '
        else:
          result = result + ch
    t=t+1
  return(result)

def treat_query(query):
  query=space(query,"∨","∨")
  query=space(query,"∧","∧")
  query=space(query,"¬","¬")
  query=query.replace("∨","or")
  query=query.replace("∧","and")
  query=query.replace("¬","not")
  query=space(query,"(",")")
  query = query.lower()
  query = query.split()
  ps = PorterStemmer()
  boolquery = [ps.stem(word) for word in query]
  return(boolquery)



In [13]:
query= "(preliminary∨(report∧time)) ∧ ¬computer "
#query=input ("Enter your boolean query :")
boolquery=treat_query(query)

words=[]
for m in boolquery:
  if m!="or" and m!="and" and m!="not" and m!="(" and m!=")" :
    words.append(m)

corpusbool=inverse_doc_freq(Data)
corpusbool=inverted_index(corpusbool)

querydictionary={"doc_1":{}}                  #Initialize a dictionary representing the presence or absence of each word from the query in each document.
for i in range (1,len(Data)+1):
  querydictionary["doc_"+str(i)] = list()

for word in words:
  listedocs=[]
  if word in corpusbool.keys():
    for n in corpusbool[word].keys():
       listedocs.append(n)

  for doc in querydictionary:
    if doc in listedocs:
      querydictionary[doc].append(1)
    else:
      querydictionary[doc].append(0)

#print (querydictionary)

result=[]
for i in range (1,len(Data)):
  boolquery2=boolquery
  k=0; c=0
  for j in  boolquery2:
    if j!="or" and j!="and" and j!="not"and j!="("and j!=")":
      boolquery2[c]=querydictionary["doc_"+str(i)][k]
      k=k+1
    c=c+1

  boolquery2=' '.join(map(str,boolquery2))
  res=eval(boolquery2)
  if res== True:
    res=1
  if res == False:
    res=0
  result.append(res)

#Display the pertinant documents
cnt=0
listdocsol=[]
for r in result:
  cnt=cnt+1
  if r==1:
    listdocsol.append("doc_"+str(cnt))
print(listdocsol)

['doc_1', 'doc_637', 'doc_2050', 'doc_2380', 'doc_2556', 'doc_2584', 'doc_2911', 'doc_2929', 'doc_2970', 'doc_2972', 'doc_3001']


# Vectorial Model

In [14]:
def freq(term , doc , data):
  return (data.get(doc)).get(term)


def max_freq(doc , data):
  dictio_terme = data.get(doc)
  values = dictio_terme.values()
  return max(values)


def get_nb_docs_contains_term(term , inv_index):
  return len(inv_index.get(term))


def compute_tf_idf(term, doc , data , inv_index):
  N = len(data)
  frequence = freq(term , doc , data)
  max_frequence = max_freq(doc , data)
  ni = get_nb_docs_contains_term(term , inv_index)
  weight = (frequence / max_frequence) * math.log((N / ni) + 1)
  return weight


def compute_ponderation_dataset(data):
  N = len(data)
  inv_index = inverted_index(inverse_doc_freq(data))
  for term in inv_index:
    dico = inv_index.get(term)
    for doc in dico :
      dico[doc] = compute_tf_idf(term , doc , data , inv_index)
  return inv_index

In [15]:
def query_terms_list(query , list_terms):
  #Return a list of all the termn with their frequency in the query
  query_dictio = {}
  query = query.split()
  tab_req = [word for word in query]
  for term in list_terms:
    if term in tab_req:
      query_dictio[term] = 1
    else:
      query_dictio[term] = 0
  return query_dictio


def inner_product_similarity(dict_query, dict_term):
  result = 0
  dict_query_values = list(dict_query.values())
  dict_term_values = list(dict_term.values())
  for i in range(len(dict_query.values())):
    result = result + dict_query_values[i] * dict_term_values[i]
  return result


def dice_similarity(dict_query, dict_term):
  tmp1 = 0; tmp2 = 0; tmp3 = 0
  dict_query_values = list(dict_query.values())
  dict_term_values = list(dict_term.values())
  for i in range(len(dict_query.values())):
    tmp1 = tmp1 + dict_query_values[i] * dict_term_values[i]
    tmp2 = tmp2 + dict_query_values[i]**2
    tmp3 = tmp3 + dict_term_values[i]**2
  return (2 * tmp1) / (tmp2 + tmp3)


def cosine_similarity(dict_query, dict_term):
  tmp1 = 0; tmp2 = 0; tmp3 = 0
  dict_query_values = list(dict_query.values())
  dict_term_values = list(dict_term.values())
  for i in range(len(dict_query.values())):
    tmp1 = tmp1 + dict_query_values[i] * dict_term_values[i]
    tmp2 = tmp2 + dict_query_values[i]**2
    tmp3 = tmp3 + dict_term_values[i]**2
  return (tmp1) / (math.sqrt(tmp2 * tmp3))


def jaccard(dict_query, dict_term):
  tmp1 = 0; tmp2 = 0; tmp3 = 0
  dict_query_values = list(dict_query.values())
  dict_term_values = list(dict_term.values())
  for i in range(len(dict_query.values())):
    tmp1 = tmp1 + dict_query_values[i] * dict_term_values[i]
    tmp2 = tmp2 + dict_query_values[i]**2
    tmp3 = tmp3 + dict_term_values[i]**2
  return (tmp1) / (tmp2 + tmp3 - tmp1)


def compute_simiralite(doc , req_dico , sim_fonction , inv_index , data):
  #Compute the similarity between the query and a document
  N = len(data)
  dico_word_term_pondere = {}
  for word in inv_index:
    dico_word_term_pondere[word] = 0
    if doc in inv_index.get(word):
      dico_word_term_pondere[word] = (inv_index.get(word)).get(doc)
  if sim_fonction == 'product':
    sim = inner_product_similarity(req_dico , dico_word_term_pondere)
  elif sim_fonction == 'cosine':
    sim = cosine_similarity(req_dico , dico_word_term_pondere)
  elif sim_fonction == 'dice':
    sim = dice_similarity(req_dico , dico_word_term_pondere)
  elif sim_fonction == 'jaccard':
    sim = jaccard(req_dico , dico_word_term_pondere)
  else:
    raise ValueError("Choose between: 'product', 'dice', 'cosine', 'jaccard'")
  return sim

In [19]:
inv_index=compute_ponderation_dataset(Data)
#Test
compute_simiralite('doc_7' , query_terms_list(preprocessing('computer report',stoplist_path) , list(inv_index.keys())) ,'cosine' ,inv_index ,Data)

0.11376074532489822

In [20]:
def get_result(query , list_doc , sim_fonction , data , inv_index , max_doc):
  # Treat the input query and return the most pertinent document, the number of returned documents is max_doc
  N = len(data)
  dico_doc_sim = {}
  query=preprocessing(query,stoplist_path)
  query_dico = query_terms_list(query , list(inv_index.keys()))
  inv_index = compute_ponderation_dataset(data)
  for doc in list_doc:
    dico_doc_sim[doc] = compute_simiralite(doc , query_dico , sim_fonction , inv_index , data)

  dico_doc_sim = sorted(dico_doc_sim.items() , key = lambda x : x[1] , reverse=True)
  return dico_doc_sim[0:max_doc]

In [21]:
# req = 'computer report'
req = 'What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers?'
get_result(req , list(Data.keys()) , 'cosine' , Data , inv_index , 10)

[('doc_1071', 0.3785051570818026),
 ('doc_1938', 0.3594777416220278),
 ('doc_1572', 0.30978738955327717),
 ('doc_2371', 0.3066996837368273),
 ('doc_2319', 0.2711126145055777),
 ('doc_971', 0.2689840210947489),
 ('doc_1908', 0.25536641514904174),
 ('doc_2151', 0.2371598807532838),
 ('doc_1680', 0.23493782943921554),
 ('doc_1657', 0.2339894061534857)]

## Vectorial Model Evaluation


In [22]:
def make_query_corpus(pathfile):
  f = open(pathfile , 'r')
  text = f.read()
  tab = re.split(pattern="\.I [0-9]+\n", string=text)
  tab.remove('')
  regex_abstract = r"\.W\n((.|\n)*?)\.[AN]\n"
  regex_author = r"\.A\n((.|\n)*?)\.N\n"
  result = []
  for string in tab:
    author = apply_regex(regex_author , string)
    abstract = apply_regex(regex_abstract , string)
    concat =  abstract + ' ' + author
    result.append(concat)
  return result

In [23]:
tab = make_query_corpus(query_path)
for i in range(len(tab)):
  tab[i] = re.sub('\n',' ',tab[i])
  tab[i] = tab[i].strip()
print(tab)

with open(qrels_path, 'r') as f:
	qfile = f.read().splitlines()
print(qfile)

dictio = {}
for f in qfile:
  t = f.split()
  key = int(t[0])
  if not key in dictio:
    dictio[key] = []
  dictio[key].append('doc_'+str(int(t[1])))
print(dictio)

['What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers?', 'I am interested in articles written either by Prieve or Udo Pooch  Prieve, B. Pooch, U.', 'Intermediate languages used in construction of multi-targeted compilers; TCOLL', "I'm interested in mechanisms for communicating between disjoint processes, possibly, but not exclusively, in a distributed environment.  I would rather see descriptions of complete mechanisms, with or without implementations, as opposed to theoretical work on the abstract problem.  Remote procedure calls and message-passing are examples of my interests.", "I'd like papers on design and implementation of editing interfaces, window-managers, command interpreters, etc.  The essential issues are human interface design, with views on improvements to user efficiency, effectiveness and satisfaction.", 'Interested in articles on robotics, motion planning particularly the geometric and combinatorial aspects.  We are not

In [ ]:
tab_f1_score = []
num=0
for x in dictio:
  num=num+1
  y_true = dictio.get(x)
  query = tab[x - 1]
  y_pred = get_result(query , list(Data.keys()) , 'cosine' , Data , inv_index , len(dictio.get(x)))
  y_pred_tab=[]
  for i in range(len(y_pred)):
    y_pred_tab.append(y_pred[i][0])

  true_positive = 0
  for doc in y_pred_tab:
    if doc in y_true:
      true_positive = true_positive + 1

  false_positive = 0
  for doc in y_pred_tab:
    if not doc in y_true:
      false_positive = false_positive + 1

  false_negative = 0
  for doc in y_true:
    if not doc in y_pred_tab:
      false_negative = false_negative + 1

  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  if precision==0 and recall==0:
    f1=0
  else:
    f1 = 2 * precision * recall / (precision + recall)

  print(num,": F1-Score:",f1)
  tab_f1_score.append(f1)

1 : F1-Score: 0.20000000000000004
2 : F1-Score: 0
3 : F1-Score: 0.16666666666666666
4 : F1-Score: 0.25
5 : F1-Score: 0
6 : F1-Score: 0
7 : F1-Score: 0.2857142857142857
8 : F1-Score: 0.3333333333333333
9 : F1-Score: 0.4444444444444444
10 : F1-Score: 0.37142857142857144
11 : F1-Score: 0.5263157894736842
12 : F1-Score: 0.20000000000000004
13 : F1-Score: 0.2727272727272727
14 : F1-Score: 0.5
15 : F1-Score: 0.20000000000000004
16 : F1-Score: 0.11764705882352941
17 : F1-Score: 0.125
18 : F1-Score: 0.18181818181818182
19 : F1-Score: 0.45454545454545453
20 : F1-Score: 0
21 : F1-Score: 0.09090909090909091
22 : F1-Score: 0.47058823529411764
23 : F1-Score: 0
24 : F1-Score: 0.15384615384615385
25 : F1-Score: 0.3333333333333333
26 : F1-Score: 0.3333333333333333
27 : F1-Score: 0.3103448275862069
28 : F1-Score: 0.6
29 : F1-Score: 0.7368421052631579
30 : F1-Score: 0.25
31 : F1-Score: 0
32 : F1-Score: 0.6666666666666666
33 : F1-Score: 0
34 : F1-Score: 0.25
35 : F1-Score: 0.16666666666666666
36 : F1-Sco

In [ ]:
print(statistics.mean(tab_f1_score))

0.26395947733281927
